In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data,DataLoader
from torch_geometric.datasets import Planetoid, LRGBDataset
import scipy.sparse as sp
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import eigh
from tqdm.notebook import tqdm
from torch_geometric.nn import GCNConv, global_mean_pool

## Cora

In [ ]:
# Laplacian Positional Encoding (LapPE)
def laplacian_positional_encoding(edge_index, num_nodes, k=10):
    edge_index = edge_index.cpu().numpy()
    row, col = edge_index[0], edge_index[1]
    adj = sp.coo_matrix((np.ones(len(row)), (row, col)), shape=(num_nodes, num_nodes))
    degree = sp.diags(adj.sum(axis=1).A1)
    
    # Compute Laplacian
    laplacian = degree - adj
    eigenvalues, eigenvectors = eigh(laplacian.toarray())
    
    # Return top k eigenvectors as LapPE
    return torch.tensor(eigenvectors[:, :k], dtype=torch.float)

# SignNet Implementation
class SignNet(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(SignNet, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x) + self.linear(-x)


In [ ]:
# GNN Model with LapPE and SignNet
class GNNWithLapPE(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features, pe_dim=10, use_signnet=False):
        super(GNNWithLapPE, self).__init__()
        self.use_signnet = use_signnet
        
        # GCN Layers
        self.conv1 = GCNConv(in_features + pe_dim, hidden_features)
        self.conv2 = GCNConv(hidden_features, out_features)

        # Optional SignNet for handling sign ambiguity
        if use_signnet:
            self.signet = SignNet(pe_dim, pe_dim)
        else:
            self.signet = None

    def forward(self, x, edge_index, lap_pe):
        if self.use_signnet:
            lap_pe = self.signet(lap_pe)  # Use SignNet for LapPE
        x = torch.cat([x, lap_pe], dim=1)  # Concatenate node features with LapPE
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
# Load Cora dataset
dataset = Planetoid(root='./dataset/cora', name='Cora')
data = dataset[0]


In [ ]:
# Compute LapPE
lap_pe = laplacian_positional_encoding(data.edge_index, data.num_nodes, k=10)

In [ ]:
# Initialize Model
model = GNNWithLapPE(
    in_features=dataset.num_node_features, 
    hidden_features=64, 
    out_features=dataset.num_classes, 
    pe_dim=10, 
    use_signnet=False  # Set this to False to disable SignNet
).to('cuda')

# Training Setup
data = data.to('cuda')
lap_pe = lap_pe.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training Loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, lap_pe)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Test Function
def test():
    model.eval()
    out = model(data.x, data.edge_index, lap_pe)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
    acc = correct / data.test_mask.sum().item()
    return acc

In [ ]:
# Training the Model
for epoch in range(100):
    loss = train()
    test_acc = test()
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [ ]:
# GNN Model with LapPE and SignNet
class GNNWithLapPE(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features, pe_dim=10, use_signnet=False):
        super(GNNWithLapPE, self).__init__()
        self.use_signnet = use_signnet
        
        # GCN Layers
        self.conv1 = GCNConv(in_features + pe_dim, hidden_features)
        self.conv2 = GCNConv(hidden_features, hidden_features)
        
        # Graph-level pooling
        self.pool = global_mean_pool
        
        # Final classifier
        self.fc = torch.nn.Linear(hidden_features, out_features)

        # Optional SignNet for handling sign ambiguity
        if use_signnet:
            self.signet = SignNet(pe_dim, pe_dim)
        else:
            self.signet = None

    def forward(self, x, edge_index, batch, lap_pe):
        if self.use_signnet:
            lap_pe = self.signet(lap_pe)  # Use SignNet for LapPE
        x = torch.cat([x, lap_pe], dim=1)  # Concatenate node features with LapPE
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.pool(x, batch)  # Pooling to get graph-level embedding
        x = self.fc(x)  # Final classification
        return F.log_softmax(x, dim=1)

In [ ]:
# Dataset and Dataloader Setup
dataset = LRGBDataset(root='dataset/peptides-func', name="Peptides-func")
peptides_train = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="train")
peptides_val = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="val")
peptides_test = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="test")

batch_size = 32
train_loader = DataLoader(peptides_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(peptides_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(peptides_test, batch_size=batch_size, shuffle=False)


In [ ]:
import torch.nn.functional as F

def laplacian_positional_encoding(edge_index, num_nodes, k=10):
    edge_index = edge_index.cpu().numpy()
    row, col = edge_index[0], edge_index[1]
    adj = sp.coo_matrix((np.ones(len(row)), (row, col)), shape=(num_nodes, num_nodes))
    degree = sp.diags(adj.sum(axis=1).A1)
    
    # Compute Laplacian
    laplacian = degree - adj
    eigenvalues, eigenvectors = eigh(laplacian.toarray())
    
    # Dynamically adjust k based on the number of nodes
    k = min(k, num_nodes - 1)
    lap_pe = torch.tensor(eigenvectors[:, :k], dtype=torch.float)

    # Pad to fixed size (e.g., 10)
    if lap_pe.size(1) < 10:
        lap_pe = F.pad(lap_pe, (0, 10 - lap_pe.size(1)), "constant", 0)
    return lap_pe


In [ ]:
# Precompute LapPE for all graphs in a dataset
def add_lap_pe(dataset, pe_dim=10):
    updated_dataset = []
    for data in tqdm(dataset):
        # Compute LapPE for the graph
        lap_pe = laplacian_positional_encoding(data.edge_index, data.num_nodes, pe_dim)
        data.lap_pe = lap_pe  # Add LapPE as an attribute
        updated_dataset.append(data)
    return updated_dataset


In [ ]:
# Add LapPE to each split of the dataset
peptides_train = add_lap_pe(peptides_train, pe_dim=10)
peptides_val = add_lap_pe(peptides_val, pe_dim=10)
peptides_test = add_lap_pe(peptides_test, pe_dim=10)

batch_size = 32
train_loader = DataLoader(peptides_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(peptides_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(peptides_test, batch_size=batch_size, shuffle=False)


In [ ]:
# Initialize Model
model = GNNWithLapPE(
    in_features=dataset.num_node_features,
    hidden_features=64,
    out_features=dataset.num_classes,
    pe_dim=10,
    use_signnet=True  # Set this to False to disable SignNet
).to('cuda')

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Training and Testing Functions
def train(loader):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to('cuda')
        
        # Ensure LapPE is moved to the GPU
        lap_pe = batch.lap_pe.to('cuda')
        
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch, lap_pe)
        
        gt = batch.y.argmax(dim=1)

        loss = F.nll_loss(out, gt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def test(loader):
    model.eval()
    correct = 0
    total = 0
    for batch in loader:
        batch = batch.to('cuda')
        lap_pe = batch.lap_pe.to('cuda')
        
        with torch.no_grad():
            # out = model(batch.x, batch.edge_index, batch.batch, batch.lap_pe.to('cuda'))
            out = model(batch.x, batch.edge_index, batch.batch, lap_pe)
            pred = out.argmax(dim=1)
            gt = batch.y.argmax(dim=1)
            
            # y = batch.y.squeeze(-1) if batch.y.dim() > 1 else batch.y
            # print(y.shape, out.shape, pred.shape)
            
            correct += (pred == gt).sum().item()
            total += batch.y.size(0)
    return correct / total

In [ ]:
# Training Loop
for epoch in range(100):
    train_loss = train(train_loader)
    val_acc = test(val_loader)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Final Test Accuracy
test_acc = test(test_loader)
print(f"Test Accuracy: {test_acc:.4f}")

## Graph Transformer

In [ ]:
def laplacian_positional_encoding(edge_index, num_nodes, k=10):
    num_nodes = int(num_nodes)  # Ensure num_nodes is an integer
    edge_index = edge_index.cpu().numpy()  # Ensure edge_index is on CPU for scipy
    row, col = edge_index[0], edge_index[1]
    adj = sp.coo_matrix((np.ones(len(row)), (row, col)), shape=(num_nodes, num_nodes))
    degree = sp.diags(adj.sum(axis=1).A1)

    # Compute Laplacian
    laplacian = degree - adj
    eigenvalues, eigenvectors = eigh(laplacian.toarray())

    # Dynamically adjust k based on the number of nodes
    k = min(k, num_nodes - 1)
    lap_pe = torch.tensor(eigenvectors[:, :k], dtype=torch.float)

    # Pad to fixed size (e.g., 10)
    if lap_pe.size(1) < 10:
        lap_pe = F.pad(lap_pe, (0, 10 - lap_pe.size(1)), "constant", 0)
    return lap_pe

# SignNet Implementation
class SignNet(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(SignNet, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x) + self.linear(-x)


In [ ]:
def add_lap_pe(dataset, pe_dim=10):
    updated_dataset = []
    for data in tqdm(dataset):
        # Compute LapPE for the graph
        lap_pe = laplacian_positional_encoding(data.edge_index, data.num_nodes, pe_dim)
        data.lap_pe = lap_pe  # Add LapPE as an attribute
        updated_dataset.append(data)
    return updated_dataset

# Dataset and Dataloader Setup
dataset = LRGBDataset(root='dataset/peptides-func', name="Peptides-func")
peptides_train = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="train")
peptides_val = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="val")
peptides_test = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="test")

# Precompute for all datasets
peptides_train = add_lap_pe(peptides_train, pe_dim=10)
peptides_val = add_lap_pe(peptides_val, pe_dim=10)
peptides_test = add_lap_pe(peptides_test, pe_dim=10)

batch_size = 32
train_loader = DataLoader(peptides_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(peptides_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(peptides_test, batch_size=batch_size, shuffle=False)


In [ ]:
from torch_geometric.nn import TransformerConv

class GraphTransformerWithLapPE(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features, pe_dim=10, use_signnet=False):
        super(GraphTransformerWithLapPE, self).__init__()
        self.use_signnet = use_signnet
        self.pe_dim = pe_dim

        # Transformer Layers
        self.transformer1 = TransformerConv(in_features + pe_dim, hidden_features, heads=4, concat=True)
        self.transformer2 = TransformerConv(hidden_features * 4, hidden_features, heads=4, concat=False)

        # Graph-level pooling
        self.pool = global_mean_pool

        # Final classifier
        self.fc = torch.nn.Linear(hidden_features, out_features)

        # Optional SignNet for handling sign ambiguity
        if self.use_signnet:
            self.signet = SignNet(pe_dim, pe_dim)
        else:
            self.signet = None

    def forward(self, x, edge_index, batch, lap_pe):
        if self.use_signnet:
            lap_pe = self.signet(lap_pe)
        lap_pe = lap_pe.to(x.device)  # Ensure LapPE is on the same device

        x = torch.cat([x, lap_pe], dim=1)  # Concatenate node features with LapPE
        x = F.relu(self.transformer1(x, edge_index))
        x = F.relu(self.transformer2(x, edge_index))
        x = self.pool(x, batch)  # Pooling to get graph-level embedding
        x = self.fc(x)  # Final classification
        return F.log_softmax(x, dim=1)


In [ ]:
# Optimizer
transformer_model = GraphTransformerWithLapPE(
    in_features=dataset.num_node_features, 
    hidden_features=8, 
    out_features=dataset.num_classes, 
    pe_dim=10, 
    use_signnet=True  # Set this to False to disable SignNet
).to('cuda')
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=0.001, weight_decay=1e-4)

def train(loader, tf_model):
    tf_model.train()
    total_loss = 0
    for batch in tqdm(loader):
        batch = batch.to('cuda')
        
        optimizer.zero_grad()
        # out = tf_model(batch.x, batch.edge_index, batch.batch, batch.num_nodes)
        out = tf_model(batch.x, batch.edge_index, batch.batch, batch.lap_pe.to('cuda'))

        gt = batch.y.argmax(dim=1)
        loss = F.nll_loss(out, gt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def test(loader, tf_model):
    tf_model.eval()
    correct = 0
    total = 0
    for batch in tqdm(loader):
        batch = batch.to('cuda')
        with torch.no_grad():
            # out = tf_model(batch.x, batch.edge_index, batch.batch, batch.num_nodes)
            out = tf_model(batch.x, batch.edge_index, batch.batch, batch.lap_pe.to('cuda'))

            pred = out.argmax(dim=1)
            gt = batch.y.argmax(dim=1)
            correct += (pred == gt).sum().item()
            total += batch.y.size(0)
    return correct / total


In [ ]:
# Training Loop
for epoch in range(10):
    train_loss = train(train_loader, transformer_model)
    val_acc = test(val_loader, transformer_model)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Final Test Accuracy
test_acc = test(test_loader, transformer_model)
print(f"Test Accuracy: {test_acc:.4f}")

## Self implemented Graph Transformer

In [2]:
def laplacian_positional_encoding(edge_index, num_nodes, k=10):
    num_nodes = int(num_nodes)  # Ensure num_nodes is an integer
    edge_index = edge_index.cpu().numpy()  # Ensure edge_index is on CPU for scipy
    row, col = edge_index[0], edge_index[1]
    adj = sp.coo_matrix((np.ones(len(row)), (row, col)), shape=(num_nodes, num_nodes))
    degree = sp.diags(adj.sum(axis=1).A1)

    # Compute Laplacian
    laplacian = degree - adj
    eigenvalues, eigenvectors = eigh(laplacian.toarray())

    # Dynamically adjust k based on the number of nodes
    k = min(k, num_nodes - 1)
    lap_pe = torch.tensor(eigenvectors[:, :k], dtype=torch.float)

    # Pad to fixed size (e.g., 10)
    if lap_pe.size(1) < 10:
        lap_pe = F.pad(lap_pe, (0, 10 - lap_pe.size(1)), "constant", 0)
    return lap_pe

# SignNet Implementation
class SignNet(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(SignNet, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x) + self.linear(-x)


In [3]:
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data

In [4]:
from torch_geometric.loader import DataLoader
from tqdm import tqdm

from torch_geometric.data import Data

class MyData(Data):
    def __cat_dim__(self, key, value, *args, **kwargs):
        if key == 'lap_pos_enc':
            return 0  # Concatenate along the node dimension
        else:
            return super().__cat_dim__(key, value, *args, **kwargs)

    def __inc__(self, key, value, *args, **kwargs):
        if key == 'lap_pos_enc':
            return 0  # No increment needed for lap_pos_enc
        else:
            return super().__inc__(key, value, *args, **kwargs)


def add_lap_pe(dataset, pe_dim=10):
    new_dataset = []
    for data in tqdm(dataset):
        try:
            lap_pe = laplacian_positional_encoding(data.edge_index, data.num_nodes, pe_dim)
            # Ensure data is an instance of MyData
            data = MyData.from_dict(data.to_dict())
            data.lap_pos_enc = lap_pe  # Add LapPE as an attribute
            new_dataset.append(data)
        except Exception as e:
            print(f"Error computing LapPE for a graph: {e}")
    return new_dataset  # Return the modified dataset


# Dataset and Dataloader Setup
dataset = LRGBDataset(root='dataset/peptides-func', name="Peptides-func")
peptides_train = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="train")
peptides_val = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="val")
peptides_test = LRGBDataset(root='dataset/peptides-func', name="Peptides-func", split="test")

# Precompute LapPE for all datasets
peptides_train = add_lap_pe(peptides_train, pe_dim=10)
peptides_val = add_lap_pe(peptides_val, pe_dim=10)
peptides_test = add_lap_pe(peptides_test, pe_dim=10)

c:\Users\Akshat\anaconda3\envs\lua_torch\lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load

In [9]:
# Dataloader Initialization
batch_size = 32
train_loader = DataLoader(peptides_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(peptides_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(peptides_test, batch_size=batch_size, shuffle=False)


In [10]:
import torch.nn as nn

In [11]:
class GraphTransformerLayer(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads=4, dropout=0.1):
        super(GraphTransformerLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim=in_dim, num_heads=num_heads, dropout=dropout)
        self.linear1 = nn.Linear(in_dim, out_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(out_dim, in_dim)
        self.norm1 = nn.LayerNorm(in_dim)
        self.norm2 = nn.LayerNorm(in_dim)
        self.activation = nn.ReLU()

    def forward(self, x, key_padding_mask=None):
        # Self-attention
        attn_output, _ = self.self_attn(x, x, x, key_padding_mask=key_padding_mask)
        x = x + attn_output
        x = self.norm1(x)

        # Feedforward layer
        linear_output = self.linear2(self.dropout(self.activation(self.linear1(x))))
        x = x + linear_output
        x = self.norm2(x)
        return x

class GraphTransformer(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads=4, num_layers=2, pe_dim=10, dropout=0.1):
        super(GraphTransformer, self).__init__()
        self.pe_dim = pe_dim

        # Input projection: Combine node features and precomputed LapPE
        self.input_proj = nn.Linear(in_dim + pe_dim, hidden_dim)

        # Stack multiple GraphTransformer layers
        self.layers = nn.ModuleList(
            [GraphTransformerLayer(hidden_dim, hidden_dim, num_heads, dropout) for _ in range(num_layers)]
        )

        # Graph-level pooling and classification
        self.pool = global_mean_pool
        self.fc = nn.Linear(hidden_dim, out_dim)

    def forward(self, x, batch, lap_pe):
        """
        Forward pass for the GraphTransformer.

        Args:
            x (Tensor): Node features [num_nodes, in_dim]
            edge_index (Tensor): Edge indices [2, num_edges]
            batch (Tensor): Batch indices for pooling [num_nodes]
            lap_pe (Tensor): Precomputed LapPE for each node [num_nodes, pe_dim]

        Returns:
            Tensor: Logits for graph-level classification [batch_size, out_dim]
        """
        # Concatenate node features and precomputed LapPE
        x = torch.cat([x, lap_pe], dim=1)

        # Project to the transformer embedding space
        x = self.input_proj(x)

        # Apply stacked GraphTransformer layers
        for layer in self.layers:
            x = layer(x)

        # Graph-level pooling (mean pooling)
        x = self.pool(x, batch)

        # Final classification layer
        x = self.fc(x)

        return F.log_softmax(x, dim=1)


In [13]:
model = GraphTransformer(
    in_dim=dataset.num_node_features, 
    hidden_dim=64, 
    out_dim=dataset.num_classes, 
    num_heads=4, 
    num_layers=2,  # Number of Transformer layers
    pe_dim=10, 
    dropout=0.1
).to('cuda')

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

In [14]:
def train(loader, model):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to('cuda')
        # Use precomputed LapPE from the batch
        lap_pe = batch.lap_pos_enc.to('cuda')
        
        optimizer.zero_grad()
        
        out = model(batch.x, batch.batch, lap_pe)
        gt = batch.y.argmax(dim=1)
        
        loss = F.nll_loss(out, gt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


def test(loader, model):
    model.eval()
    correct = 0
    total = 0
    for batch in loader:
        batch = batch.to('cuda')
        with torch.no_grad():
            # Use precomputed LapPE from the batch
            lap_pe = batch.lap_pos_enc.to('cuda')
            
            out = model(batch.x, batch.batch, lap_pe)
            pred = out.argmax(dim=1)
            gt = batch.y.argmax(dim=1)
            correct += (pred == gt).sum().item()
            total += batch.y.size(0)

    return correct / total


In [15]:
batch = next(iter(train_loader))

print(batch.keys)

<bound method BaseData.keys of MyDataBatch(x=[5073, 9], edge_index=[2, 10338], edge_attr=[10338, 3], y=[32, 10], lap_pos_enc=[5073, 10], batch=[5073], ptr=[33])>


In [16]:
# Training Loop
for epoch in range(10):
    train_loss = train(train_loader, model)
    val_acc = test(val_loader, model)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Final Test Accuracy
test_acc = test(test_loader, model)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1, Train Loss: 1.5369, Validation Accuracy: 0.5157
Epoch 2, Train Loss: 1.4999, Validation Accuracy: 0.5221
Epoch 3, Train Loss: 1.4280, Validation Accuracy: 0.5337
Epoch 4, Train Loss: 1.3099, Validation Accuracy: 0.6032
Epoch 5, Train Loss: 1.2143, Validation Accuracy: 0.6143
Epoch 6, Train Loss: 1.1942, Validation Accuracy: 0.6096
Epoch 7, Train Loss: 1.1776, Validation Accuracy: 0.6225
Epoch 8, Train Loss: 1.1716, Validation Accuracy: 0.6268
Epoch 9, Train Loss: 1.1654, Validation Accuracy: 0.6238
Epoch 10, Train Loss: 1.1589, Validation Accuracy: 0.6242
Test Accuracy: 0.6251
